<a href="https://colab.research.google.com/github/pilgujeong10/TEST/blob/master/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# CIFAR-10
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data

(x_train, y_train),(x_test,y_test)= load_data()

x_train = x_train/255.
x_test - x_test/255.

y_train = y_train.reshape([-1])
y_test = y_test.reshape([-1])

x= tf.placeholder(tf.float32, shape=[None,32,32,3])
y= tf.placeholder(tf.int64, shape=[None])

y_onehot = tf.one_hot(y,10)

keep_prob = tf.placeholder(tf.float32)

In [0]:
W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,3,64],stddev=5e-2))
b_conv1 = tf.Variable(tf.constant(0.1,shape=[64]))
h_conv1= tf.nn.relu(tf.nn.conv2d(x,W_conv1,strides=[1,1,1,1],padding='SAME')+b_conv1)
h_pool1= tf.nn.max_pool(h_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,64,64],stddev=5e-2))
b_conv2 = tf.Variable(tf.constant(0.1,shape=[64]))
h_conv2= tf.nn.relu(tf.nn.conv2d(h_pool1 , W_conv2 ,strides=[1,1,1,1],padding='SAME')+b_conv2)
h_pool2= tf.nn.max_pool(h_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W_fc1 = tf.Variable(tf.truncated_normal(shape=[8*8*64,384],stddev=5e-2))
b_fc1 = tf.Variable(tf.constant(0.1,shape=[384]))
h_pool2_flat = tf.reshape(h_pool2,[-1,8*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob=keep_prob)

W_fc2 = tf.Variable(tf.truncated_normal(shape=[384,10],stddev=5e-2))
b_fc2 = tf.Variable(tf.constant(0.1,shape=[10]))
logits = tf.matmul(h_fc1_drop,W_fc2)+b_fc2
y_pred = tf.nn.softmax(logits)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_onehot,logits=logits))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y_pred,1),y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for i in range(10000):
    idx = np.random.randint(x_train.shape[0],size=128)
    batch = (x_train[idx], y_train[idx])
    if i % 100 ==0:
      train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y:batch[1], keep_prob:1.})
      loss_print = loss.eval(feed_dict={x:batch[0],y:batch[1], keep_prob:1.})
      print("step: %d, acc:%f, loss:%f" % (i, train_accuracy, loss_print))
      
    sess.run(train_step, feed_dict={x:x_test,y:y_test, keep_prob:0.5})
  test_accuracy = accuracy.eval(feed_dict={x:x_test,y:y_test, keep_prob:1.})
  print("test acc : %f" %test_accuracy)
 

step: 0, acc:0.117188, loss:2.413371
step: 100, acc:0.117188, loss:2.301052
step: 200, acc:0.156250, loss:2.300285
step: 300, acc:0.226562, loss:2.296139
step: 400, acc:0.164062, loss:2.293788
step: 500, acc:0.171875, loss:2.293326
step: 600, acc:0.156250, loss:2.288141
step: 700, acc:0.226562, loss:2.285082
step: 800, acc:0.210938, loss:2.282581
step: 900, acc:0.164062, loss:2.283262
step: 1000, acc:0.218750, loss:2.271361
step: 1100, acc:0.148438, loss:2.282826
step: 1200, acc:0.132812, loss:2.291707
